# Circuit & Device Modeling

## 1. Supported Verilog-A Models

We support two kinds of Verilog-A models:
- You can either write your own Verilog-A model for the device, or
- You can fit your device to one of the pre-defined models for the following memory technologies:
    - Resistive Random Access Memory (ReRAM) [1]
    - Ferrorelectric Tunnel Junction (FTJ) [2]

[1] S. Kvatinsky, M. Ramadan, E. G. Friedman, and A. Kolodny, "VTEAM – A General Model for Voltage Controlled Memristor", Transactions on Circuits and Systems II: Express Briefs, Vol. 62, No. 8, pp. 786-790, August 2015 <br>

[2] Z. Wang, W. Zhao, W. Kang, Y. Zhang, J. O. Klein, D. Ravelosona, and C. Chappert, "Compact Modelling of Ferroelectric Tunnel Memristor and its Use for Neuromorphic Simulation," Appl. Phys. Lett., vol. 104, no. 5, pp. 053505, 2014. DOI: 10.1063/1.4864270

<center>
    <img src="different_verilogA_models.png" width="600">
</center>

### 1.1. ReRAM Memory Technology - VTEAM Model

We use the pre-defined model for ReRAM memory technology and fit it to two different ReRAM device [3, 4] as per the following parameters:

<center>
    <img src="ReRAM Model Fitting Parameter.png" width="600">
</center>

<br>

Although both the devices are of same memory technology, they differ widely in their behavior.

<center>
    <img src="High-level Overview.png" width="600">
</center>

<br>

[3] S. Kvatinsky, D. Belousov, S. Liman, G. Satat, N. Wald, E. G. Friedman, A. Kolodny, and U. C. Weiser, "MAGIC – Memristor Aided LoGIC", IEEE Transactions on Circuits and Systems II: Express Briefs, Vol. 61, No. 11, pp. 895- 899, November 2014. <br>

[4] M. Imani, S. Gupta, Y. Kim and T. Rosing, "FloatPIM: In-Memory Acceleration of Deep Neural Network Training with High Precision," 2019 ACM/IEEE 46th Annual International Symposium on Computer Architecture (ISCA), Phoenix, AZ, USA, 2019, pp. 802-815.



In [ ]:
import os
home = "/home/demo/" #your homedir
os.chdir(str(home + "Mastodon"))

### 1.2. Creating a ReRAM Verilog-A Model

In [ ]:
%%writefile ./src/device-model-plugins/device_models/ReRAM_VTEAM.va

`include "disciplines.vams"
`include "constants.h"

module Memristor(p, n, w_position);
    inout p, n;
    output w_position;
 
    electrical p, n, w_position;
 
    parameter real initial_state = 0;
    parameter real R_HRS = 300e3;
    parameter real R_LRS = 1e3;
    parameter real K_SET = -216.2;
    parameter real K_RESET = 0.091;
    parameter real Alpha_SET = 4;
    parameter real Alpha_RESET = 4;
    parameter real V_SET = -1.5;
    parameter real V_RESET = 0.3;
    parameter real IV_relation = 1;
    parameter real x_on = 0;
    parameter real x_off = 3e-09;
    parameter real D = 3e-9;

    real R;
    real first_iteration = 0;
    real x;
    real dxdt;
    real x_last;
    real l;
    real tp1;
    real t1;
    real dt;
 
    analog 
    begin
        if(first_iteration==0) 
        begin   
            x_last=(1-initial_state)*D;    
        end
    
        tp1 = $abstime;
        dt = tp1 - t1;

        if (V(p,n) <= V_SET) 
        begin
            dxdt = K_SET*pow((V(p,n)/V_SET-1), Alpha_SET);
        end
        else if (V(p,n) >= V_RESET) 
        begin 
            dxdt = K_RESET*pow((V(p,n)/V_RESET-1), Alpha_RESET);
        end
        else 
        begin
            dxdt=0;
        end

        x = x_last+dt*dxdt;
        x_last=x;
        
        if (x >= D) 
        begin
            dxdt=0;             
            x=D;
        end
        else if (x <= 0) 
        begin
            dxdt=0;
            x=0;
        end
                        
        l = ln(R_HRS/R_LRS);
            
        if (IV_relation==0) 
        begin 
            I(p,n) <+ V(p,n)/(R_HRS*x/D+R_LRS*(1-x/D));
        end
        else if (IV_relation==1) 
        begin 
            I(p,n) <+ V(p,n)/(R_LRS*exp(l*(x-x_on)/(x_off-x_on)));  
        end
    
        t1 = $abstime;
        first_iteration=1;  

        V(w_position) <+ 1 - (x/D);                               
    end                                            
endmodule

## 2. Adding Device-Level Configuration Parameters

In [ ]:
%%writefile -a ./tutorial/device/CMPEQ.config

//Device Level Parameters
device_model_sim = true
memorisation = true
verilog_filename = ReRAM_VTEAM
volt_MAGIC = -1.4
volt_SET = -1.7
volt_RESET = 0.4
volt_ISO_BL = -0.28
volt_ISO_WL = -0.3
second_cycle_time = 0.0002
second_step_size = 50e-6
ohm_R = 2.925
farad_C = 1e-15
state_threshold = 0.5

### 2.1. Simulating CMPEQ Instruction

In [ ]:
%%writefile ./tutorial/device/CMPEQ.rc

COMPUTE 0 0
CMPEQ 0 1
COMPUTE_DONE
FLUSH
EOF

In [ ]:
!./tutorial/device/init.sh #Make the init xml files needed

In [ ]:
#Change to the demo_dir
cwd = os.getcwd()
deviceDemoDir = str(cwd + "/tutorial/device/")
os.chdir(deviceDemoDir)

In [ ]:
!./run_mastodon.sh

In [ ]:
#Change back to top-level dir
os.chdir(str(home + "Mastodon"))

## 3. Effect of Cycle Time and Assertion Voltage on Data Errors

### 3.1. Changing Cycle Time

In [ ]:
filename = './tutorial/device/CMPEQ.config'

def update_config(filename, updates):
    with open(filename, 'r') as file:
        lines = file.readlines()  

    updated_lines = []
    for line in lines:
        for key, value in updates.items():
            if line.strip().startswith(key): 
                line = f'{key} = {value}\n'  
        updated_lines.append(line) 

    with open(filename, 'w') as file:
        file.writelines(updated_lines)

updates = {
           'second_cycle_time': 0.0004      ###  Changing the cycle time from 200 microsecond to 400 microsecond  ###
}

update_config(filename, updates)

In [ ]:
#Change to the demo_dir
os.chdir(deviceDemoDir)
!./run_mastodon.sh

### 3.2. Changing Assertion Voltage

In [ ]:
#Change back to top-level dir
os.chdir(str(home + "Mastodon"))

updates = {
           'volt_MAGIC': -1.5,        ###  Changing the assertion voltage from -1.40 V to -1.50 V  ###
           'volt_ISO_BL': -0.3,       ###  Changing the isolation voltage from -0.28 V to -0.30 V  ###
}

update_config(filename, updates)

In [ ]:
#Change to the demo_dir
os.chdir(deviceDemoDir)
!./run_mastodon.sh

In [ ]:
#Change back to top-level dir
os.chdir(str(home + "Mastodon"))

### 3.3. Error Analysis for CMPEQ Instruction

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

error_CMPEQ = {
    '500': {'-1.3': 0.98, '-1.4': 0.80, '-1.5': 0.00, '-1.6': 0.41},
    '400': {'-1.3': 0.98, '-1.4': 0.86, '-1.5': 0.00, '-1.6': 0.51},
    '300': {'-1.3': 0.98, '-1.4': 0.87, '-1.5': 0.09, '-1.6': 0.51},
    '200': {'-1.3': 0.98, '-1.4': 0.88, '-1.5': 0.22, '-1.6': 0.77},
    '100': {'-1.3': 0.98, '-1.4': 0.98, '-1.5': 0.98, '-1.6': 0.77}
}

# Function to convert the dictionary to a numpy array and labels for plotting
def dict_to_heatmap_data(data):
    rows = sorted(data.keys(), key=int)
    cols = sorted(next(iter(data.values())).keys(), key=float)
    heatmap_data = np.array([[data[r][c] for c in cols] for r in rows])
    return heatmap_data, rows, cols

# Function to plot heatmap using matplotlib
def plot_heatmap(ax, data, title):
    heatmap_data, row_labels, col_labels = dict_to_heatmap_data(data)
    col_labels = col_labels[::-1]
    heatmap_data = heatmap_data[:, ::-1]
    row_labels = row_labels[::-1]
    heatmap_data = heatmap_data[::-1, :]
    cax = ax.matshow(heatmap_data, cmap='RdYlGn_r', aspect='auto', vmin = 0, vmax = 1)
    ax.set_title(title)
    ax.set_xticks(np.arange(len(col_labels)))
    ax.set_xticklabels(col_labels, fontsize = 15)
    ax.set_yticks(np.arange(len(row_labels)))
    ax.set_yticklabels(row_labels, fontsize = 15)
    for i in range(len(row_labels)):
        for j in range(len(col_labels)):
            text = ax.text(j, i, f"{heatmap_data[i, j]:.2f}", fontsize = 15,
                           ha="center", va="center", color="black")
            
            rect = plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=False, edgecolor='black', linewidth=1)
            ax.add_patch(rect)
            
    ax.xaxis.set_ticks_position('bottom')
    ax.xaxis.set_label_position('bottom')
    ax.set_xlabel('Assertion Voltage (V)', fontsize = 15)
    return cax

# Create a figure and a grid of subplots
fig, axes = plt.subplots(1, 1, figsize=(6, 6))

# Plot each heatmap
cax = plot_heatmap(axes, error_CMPEQ, 'Ratio of Incorrect NOR Operations - CMPEQ')

axes.set_title('Ratio of Incorrect NOR Operations - CMPEQ', fontsize = 20, weight = "bold")
axes.set_ylabel('Cycle Time (us)', fontsize = 15)


# Adjust layout and add colorbars
fig.colorbar(cax, ax=axes)

plt.tight_layout()


plt.show()

### 3.4. Error Analysis across Multiple Instructions and Devices

In [ ]:
# Define the dictionaries
error_MUL8_device_1 = {
    '500': {'-1.3': 0.46, '-1.4': 0.37, '-1.5': 0.00, '-1.6': 0.32},
    '400': {'-1.3': 0.46, '-1.4': 0.37, '-1.5': 0.13, '-1.6': 0.32},
    '300': {'-1.3': 0.46, '-1.4': 0.44, '-1.5': 0.28, '-1.6': 0.33},
    '200': {'-1.3': 0.46, '-1.4': 0.44, '-1.5': 0.42, '-1.6': 0.32},
    '100': {'-1.3': 0.48, '-1.4': 0.46, '-1.5': 0.44, '-1.6': 0.32}
}

error_POPC8_device_1 = {
    '500': {'-1.3': 0.98, '-1.4': 0.78, '-1.5': 0.01, '-1.6': 0.41},
    '400': {'-1.3': 0.98, '-1.4': 0.78, '-1.5': 0.07, '-1.6': 0.51},
    '300': {'-1.3': 0.98, '-1.4': 0.78, '-1.5': 0.48, '-1.6': 0.51},
    '200': {'-1.3': 0.98, '-1.4': 0.92, '-1.5': 0.79, '-1.6': 0.54},
    '100': {'-1.3': 0.98, '-1.4': 0.98, '-1.5': 0.98, '-1.6': 0.54}
}

error_CMPEQ_device_1 = {
    '500': {'-1.3': 0.98, '-1.4': 0.80, '-1.5': 0.00, '-1.6': 0.41},
    '400': {'-1.3': 0.98, '-1.4': 0.86, '-1.5': 0.00, '-1.6': 0.51},
    '300': {'-1.3': 0.98, '-1.4': 0.87, '-1.5': 0.09, '-1.6': 0.51},
    '200': {'-1.3': 0.98, '-1.4': 0.88, '-1.5': 0.22, '-1.6': 0.77},
    '100': {'-1.3': 0.98, '-1.4': 0.98, '-1.5': 0.98, '-1.6': 0.77}
}

error_MUL8_device_2 = {
    '900': {'-5': 0.32, '-5.5': 0.28, '-6': 0.00, '-6.5': 0.40},
    '800': {'-5': 0.32, '-5.5': 0.29, '-6': 0.05, '-6.5': 0.40},
    '700': {'-5': 0.38, '-5.5': 0.33, '-6': 0.21, '-6.5': 0.40},
    '600': {'-5': 0.38, '-5.5': 0.39, '-6': 0.31, '-6.5': 0.42},
    '500': {'-5': 0.44, '-5.5': 0.39, '-6': 0.39, '-6.5': 0.40}
}

error_POPC8_device_2 = {
    '900': {'-5': 0.93, '-5.5': 0.79, '-6': 0.09, '-6.5': 0.88},
    '800': {'-5': 0.95, '-5.5': 0.79, '-6': 0.32, '-6.5': 0.88},
    '700': {'-5': 0.95, '-5.5': 0.80, '-6': 0.65, '-6.5': 0.89},
    '600': {'-5': 0.95, '-5.5': 0.93, '-6': 0.75, '-6.5': 0.90},
    '500': {'-5': 0.95, '-5.5': 0.98, '-6': 0.91, '-6.5': 0.91}
}

error_CMPEQ_device_2 = {
    '900': {'-5': 0.93, '-5.5': 0.75, '-6': 0.00, '-6.5': 0.85},
    '800': {'-5': 0.95, '-5.5': 0.75, '-6': 0.19, '-6.5': 0.86},
    '700': {'-5': 0.95, '-5.5': 0.78, '-6': 0.42, '-6.5': 0.86},
    '600': {'-5': 0.95, '-5.5': 0.83, '-6': 0.70, '-6.5': 0.89},
    '500': {'-5': 0.95, '-5.5': 0.91, '-6': 0.91, '-6.5': 0.92}
}

# Create a figure and a grid of subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot each heatmap
cax1 = plot_heatmap(axes[0, 0], error_MUL8_device_1, 'MUL8 Device 1')
cax2 = plot_heatmap(axes[0, 1], error_POPC8_device_1, 'POPC8 Device 1')
cax3 = plot_heatmap(axes[0, 2], error_CMPEQ_device_1, 'CMPEQ Device 1')
cax4 = plot_heatmap(axes[1, 0], error_MUL8_device_2, 'MUL8 Device 2')
cax5 = plot_heatmap(axes[1, 1], error_POPC8_device_2, 'POPC8 Device 2')
cax6 = plot_heatmap(axes[1, 2], error_CMPEQ_device_2, 'CMPEQ Device 2')

axes[0, 0].set_title('MUL8 - Device 1', fontsize = 15)
axes[0, 0].set_ylabel('Cycle Time (us)', fontsize = 15)
axes[0, 1].set_title('POPC8 - Device 1', fontsize = 15)
axes[0, 1].set_ylabel('Cycle Time (us)', fontsize = 15)
axes[0, 2].set_title('CMPEQ - Device 1', fontsize = 15)
axes[0, 2].set_ylabel('Cycle Time (us)', fontsize = 15)

axes[1, 0].set_title('MUL8 - Device 2', fontsize = 15)
axes[1, 0].set_ylabel('Cycle Time (ms)', fontsize = 15)
axes[1, 1].set_title('POPC8 - Device 2', fontsize = 15)
axes[1, 1].set_ylabel('Cycle Time (ms)', fontsize = 15)
axes[1, 2].set_title('CMPEQ - Device 2', fontsize = 15)
axes[1, 2].set_ylabel('Cycle Time (ms)', fontsize = 15)

# Adjust layout and add colorbars
fig.colorbar(cax1, ax=axes[0, 0])
fig.colorbar(cax2, ax=axes[0, 1])
fig.colorbar(cax3, ax=axes[0, 2])
fig.colorbar(cax4, ax=axes[1, 0])
fig.colorbar(cax5, ax=axes[1, 1])
fig.colorbar(cax6, ax=axes[1, 2])

plt.suptitle("Error Analysis across Multiple Instructions & Devices", fontsize=30, weight='bold')
# plt.tight_layout()
line_y_position = (axes[0, 0].get_position().y0 + axes[1, 0].get_position().y1 - 0.03) / 2
fig.add_artist(plt.Line2D((0, 1), (line_y_position, line_y_position), color='black', linestyle='--', linewidth=2))
plt.subplots_adjust(hspace=0.4)



plt.show()

In [ ]:
error_MUL8_per_tile = {'0': 0.83,
                       '1': 0.86,
                       '2': 0.89,
                       '3': 1.00,
                       '4': 0.00,
                       '5': 0.00,
                       '6': 0.00,
                       '7': 0.00}

# Convert dictionary keys and values to lists
x = list(error_MUL8_per_tile.keys())
y = list(error_MUL8_per_tile.values())

# Plot the data
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x, y, marker='o', linestyle='-', color='b')
ax.fill_between(x, 0, 1, where=[0 <= int(tile) <= 3 for tile in x], color='lightcoral', alpha=0.5)
ax.set_xlim(-0.1, 7.1)

# Add title and labels
ax.set_title('Error Per Bit for MUL8', fontsize=20, weight='bold')
ax.set_xlabel('Bit Number', fontsize=14)
ax.set_ylabel('Ratio of Incorrect NOR Operations', fontsize=14)

# Set tick font size
ax.tick_params(axis='both', which='major', labelsize=14)

# Show grid
plt.grid(True)

# Show plot
plt.show()


## 4. Discrete Fourier Transform (DFT) with ReRAM Memory Technology

In [ ]:
def read_text_file_to_array(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Convert lines of text into a 2D list of integers
    data = [list(map(int, line.split())) for line in lines]
    return np.abs(np.array(np.array(data)[1:]).T[1:])

def normalize_pixel_values(array):
    # Normalize pixel values to range [0, 255] if necessary
    min_val = np.min(array)
    max_val = np.max(array)
    if min_val < 0 or max_val > 255:
        array = 255 * (array - min_val) / (max_val - min_val)
    return array.astype(np.uint8)
    
def convert_negative(array):
    output = []
    for row in array:
        converted_row = []
        for elem in row:
            if (elem > 4294967296):
                elem = ~elem + 1
            converted_row.append(elem)
        output.append(converted_row)
    return np.asarray(output)
    
def display_image(array):
    plt.imshow(array, cmap='gray', vmin=0, vmax=255)
    plt.axis('off')  # Turn off axis numbers and ticks
    plt.show()

def display_images_side_by_side(filenames):
    images = []
    titles = ["Cycle 0", "Cycle 20000", "Cycle 40000", "Cycle 60000", "Cycle 80000", "Cycle 100000", "Cycle 120000", "Cycle 140000"]

    for filename in filenames:
        array = read_text_file_to_array(filename)
        array = normalize_pixel_values(array)
        array = convert_negative(array)
        images.append(array)
    
    # Display images in a 2x4 grid (last cell empty)
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i], cmap='gray', vmin=0, vmax=255)
            ax.set_title(titles[i], fontsize=20)
        ax.axis('off')
    
    # Add a single title for the entire figure
    plt.suptitle("Evolution of DFT Matrix", fontsize=30, weight='bold')
    # plt.tight_layout()
    plt.show()

# List of filenames
filenames = [
    './tutorial/device/dft_mat/dft_mat_0.txt',
    './tutorial/device/dft_mat/dft_mat_20000.txt',
    './tutorial/device/dft_mat/dft_mat_40000.txt',
    './tutorial/device/dft_mat/dft_mat_60000.txt',
    './tutorial/device/dft_mat/dft_mat_80000.txt',
    './tutorial/device/dft_mat/dft_mat_100000.txt',
    './tutorial/device/dft_mat/dft_mat_120000.txt',
    './tutorial/device/dft_mat/dft_mat_140000.txt'
]

display_images_side_by_side(filenames)